In [21]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
file_path = '/content/drive/MyDrive/prueba_encoder/alpha2_dataset(2).csv'
df = pd.read_csv(file_path)
df = df.fillna(pd.NA)

print(df.head())
df.info()

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [23]:
# stemmer, lemmatizer and stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from typing import Optional

# Initialize NLTK resources
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'  # Adjective
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun if not recognized

def remove_extra_new_lines(text):

    if pd.isnull(text):  # check if text is nan
        return ''  # replace with an empty string

    clean_text = [i for i in str(text).splitlines() if i.strip()]
    clean_text = ' '.join(clean_text)
    return clean_text

def remove_extra_whitespace(text: str) -> str:

    spaceless_text = re.sub(r'\s+', ' ', text)
    return spaceless_text

def remove_special_chars(text: str, remove_digits: Optional[bool] = False) -> str:

    if remove_digits:
        pattern = r'[^a-zA-Z\s]'
    else:
        pattern = r'[^a-zA-Z0-9\s]'

    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def normalize_text(text):

    text = remove_extra_new_lines(text)

    text = remove_extra_whitespace(text)

    text = remove_special_chars(text, remove_digits=False)

    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    tagged_tokens = pos_tag(tokens)
    lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in tagged_tokens]

    return ' '.join(lemmas)

In [24]:
normalization = ['name', 'description']
for column in normalization:
    df[column + '_normalized'] = df[column].apply(normalize_text)

print(df.shape)

(50041, 13)


In [25]:
df['sub_category_1'].fillna('0', inplace=True)
df['sub_category_2'].fillna('0', inplace=True)
df['sub_category_3'].fillna('0', inplace=True)
df['sub_category_4'].fillna('0', inplace=True)


In [26]:
print(df.head())
df.shape
X = df.drop(columns=['parent_category'])
y = df['parent_category']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [27]:
df.shape
X_1 = df.drop(columns=['sub_category_1'])
y_1 = df['sub_category_1']

label_encoder = LabelEncoder()
y_1_encoded = label_encoder.fit_transform(y)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1_encoded, test_size=0.2, random_state=42)

In [28]:
df.shape
X_2 = df.drop(columns=['sub_category_2'])
y_2 = df['sub_category_2']

label_encoder = LabelEncoder()
y_2_encoded = label_encoder.fit_transform(y)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2_encoded, test_size=0.2, random_state=42)

In [29]:
df.shape
X_3 = df.drop(columns=['sub_category_3'])
y_3 = df['sub_category_3']

label_encoder = LabelEncoder()
y_3_encoded = label_encoder.fit_transform(y)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3_encoded, test_size=0.2, random_state=42)

In [30]:
df.shape
X_4 = df.drop(columns=['sub_category_4'])
y_4 = df['sub_category_4']

label_encoder = LabelEncoder()
y_4_encoded = label_encoder.fit_transform(y)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_4, y_4_encoded, test_size=0.2, random_state=42)

In [ ]:
print(df.head())
df.shape

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


(50041, 13)

In [20]:
print(X_train.shape)
print( X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_train['name_normalized'])

(40032, 12)
(10009, 12)
(40032,)
(10009,)
9233                            star fox preowned nintendo
25631    pioneer networkready ultra hd passthrough av h...
19030                     evolve ultimate edition xbox one
12044    joby pro series ultraplate quickrelease plate ...
18967    aluratek bump w home audio speaker system ipod...
                               ...                        
11284    samsung class diag lead curved smart ultra hd ...
44732    hifonics brutus class mono mosfet subwoofer am...
38158    mobile edge premium laptop backpack apple macb...
860                                 presonus presonus gray
15795      insignia portable bluetooth stereo speaker blue
Name: name_normalized, Length: 40032, dtype: object


In [ ]:
y_train

array([299, 299, 299, ..., 299, 299, 299])

One Hot Encoder y Scaler


Stage 1

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Define columns
categorical_columns = ['type', 'manufacturer']
numerical_columns = ['price']
text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train[categorical_columns] = X_train[categorical_columns].fillna('missing')
X_test[categorical_columns] = X_test[categorical_columns].fillna('missing')

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[categorical_columns])
X_test_encoded = encoder.transform(X_test[categorical_columns])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded:", X_train_encoded.shape)
print("Data type of X_train_encoded:", type(X_train_encoded))
print("Data type of elements in X_train_encoded:", X_train_encoded.dtype)
print("Shape of X_test_encoded:", X_test_encoded.shape)
print("Data type of X_test_encoded:", type(X_test_encoded))
print("Data type of elements in X_test_encoded:", X_test_encoded.dtype)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numerical_columns])
X_test_scaled = scaler.transform(X_test[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Data type of elements in X_train_scaled:", X_train_scaled.dtype)
print("Shape of X_test_scaled:", X_test_scaled.shape)
print("Data type of elements in X_test_scaled:", X_test_scaled.dtype)

Shape of X_train_encoded: (40032, 2195)
Data type of X_train_encoded: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded: float64
Shape of X_test_encoded: (10009, 2195)
Data type of X_test_encoded: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded: float64
Shape of X_train_scaled: (40032, 1)
Data type of elements in X_train_scaled: float64
Shape of X_test_scaled: (10009, 1)
Data type of elements in X_test_scaled: float64


Stage 2

In [33]:
# Define columns
categorical_columns_1 = ['type', 'manufacturer', 'parent_category']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_1[categorical_columns_1] = X_train_1[categorical_columns_1].fillna('missing')
X_test_1[categorical_columns_1] = X_test_1[categorical_columns_1].fillna('missing')

# One-hot encode categorical features
encoder_1 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_1 = encoder_1.fit_transform(X_train_1[categorical_columns_1])
X_test_encoded_1 = encoder_1.transform(X_test_1[categorical_columns_1])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_1:", X_train_encoded_1.shape)
print("Data type of X_train_encoded_1:", type(X_train_encoded_1))
print("Data type of elements in X_train_encoded_1:", X_train_encoded_1.dtype)
print("Shape of X_test_encoded_1:", X_test_encoded_1.shape)
print("Data type of X_test_encoded_1:", type(X_test_encoded_1))
print("Data type of elements in X_test_encoded_1:", X_test_encoded_1.dtype)

# Scale numerical features
scaler_1 = StandardScaler()
X_train_scaled_1 = scaler_1.fit_transform(X_train_1[numerical_columns])
X_test_scaled_1 = scaler_1.transform(X_test_1[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_1:", X_train_scaled_1.shape)
print("Data type of elements in X_train_scaled_1:", X_train_scaled_1.dtype)
print("Shape of X_test_scaled_1:", X_test_scaled_1.shape)
print("Data type of elements in X_test_scaled_1:", X_test_scaled_1.dtype)

Shape of X_train_encoded_1: (40032, 2253)
Data type of X_train_encoded_1: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_1: float64
Shape of X_test_encoded_1: (10009, 2253)
Data type of X_test_encoded_1: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_1: float64
Shape of X_train_scaled_1: (40032, 1)
Data type of elements in X_train_scaled_1: float64
Shape of X_test_scaled_1: (10009, 1)
Data type of elements in X_test_scaled_1: float64


Stage 3

In [34]:
# Define columns
categorical_columns_2 = ['type', 'manufacturer', 'parent_category', 'sub_category_1']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_2[categorical_columns_2] = X_train_2[categorical_columns_2].fillna('missing')
X_test_2[categorical_columns_2] = X_test_2[categorical_columns_2].fillna('missing')

# One-hot encode categorical features
encoder_2 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_2 = encoder_2.fit_transform(X_train_2[categorical_columns_2])
X_test_encoded_2 = encoder_2.transform(X_test_2[categorical_columns_2])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_2:", X_train_encoded_2.shape)
print("Data type of X_train_encoded_2:", type(X_train_encoded_2))
print("Data type of elements in X_train_encoded_2:", X_train_encoded_2.dtype)
print("Shape of X_test_encoded_2:", X_test_encoded_2.shape)
print("Data type of X_test_encoded_2:", type(X_test_encoded_2))
print("Data type of elements in X_test_encoded_2:", X_test_encoded_2.dtype)

# Scale numerical features
scaler_2 = StandardScaler()
X_train_scaled_2 = scaler_2.fit_transform(X_train_2[numerical_columns])
X_test_scaled_2 = scaler_2.transform(X_test_2[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_2:", X_train_scaled_2.shape)
print("Data type of elements in X_train_scaled_2:", X_train_scaled_2.dtype)
print("Shape of X_test_scaled_2:", X_test_scaled_2.shape)
print("Data type of elements in X_test_scaled_2:", X_test_scaled_2.dtype)

Shape of X_train_encoded_2: (40032, 2398)
Data type of X_train_encoded_2: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_2: float64
Shape of X_test_encoded_2: (10009, 2398)
Data type of X_test_encoded_2: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_2: float64
Shape of X_train_scaled_2: (40032, 1)
Data type of elements in X_train_scaled_2: float64
Shape of X_test_scaled_2: (10009, 1)
Data type of elements in X_test_scaled_2: float64


Stage 4

In [35]:
# Define columns
categorical_columns_3 = ['type', 'manufacturer', 'parent_category', 'sub_category_1', 'sub_category_2']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_3[categorical_columns_3] = X_train_3[categorical_columns_3].fillna('missing')
X_test_3[categorical_columns_3] = X_test_3[categorical_columns_3].fillna('missing')

# One-hot encode categorical features
encoder_3 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_3 = encoder_3.fit_transform(X_train_3[categorical_columns_3])
X_test_encoded_3 = encoder_3.transform(X_test_3[categorical_columns_3])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_3:", X_train_encoded_3.shape)
print("Data type of X_train_encoded_3:", type(X_train_encoded_3))
print("Data type of elements in X_train_encoded_3:", X_train_encoded_3.dtype)
print("Shape of X_test_encoded_3:", X_test_encoded_3.shape)
print("Data type of X_test_encoded_3:", type(X_test_encoded_3))
print("Data type of elements in X_test_encoded_3:", X_test_encoded_3.dtype)

# Scale numerical features
scaler_3 = StandardScaler()
X_train_scaled_3 = scaler_3.fit_transform(X_train_3[numerical_columns])
X_test_scaled_3 = scaler_3.transform(X_test_3[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_3:", X_train_scaled_3.shape)
print("Data type of elements in X_train_scaled_3:", X_train_scaled_3.dtype)
print("Shape of X_test_scaled_3:", X_test_scaled_3.shape)
print("Data type of elements in X_test_scaled_3:", X_test_scaled_3.dtype)

Shape of X_train_encoded_3: (40032, 2943)
Data type of X_train_encoded_3: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_3: float64
Shape of X_test_encoded_3: (10009, 2943)
Data type of X_test_encoded_3: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_3: float64
Shape of X_train_scaled_3: (40032, 1)
Data type of elements in X_train_scaled_3: float64
Shape of X_test_scaled_3: (10009, 1)
Data type of elements in X_test_scaled_3: float64


Stage 5

In [37]:
# Define columns
categorical_columns_4 = ['type', 'manufacturer', 'parent_category', 'sub_category_1', 'sub_category_2', 'sub_category_3']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_4[categorical_columns_4] = X_train_4[categorical_columns_4].fillna('missing')
X_test_4[categorical_columns_4] = X_test_4[categorical_columns_4].fillna('missing')

# One-hot encode categorical features
encoder_4 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_4 = encoder_4.fit_transform(X_train_4[categorical_columns_4])
X_test_encoded_4 = encoder_4.transform(X_test_4[categorical_columns_4])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_4:", X_train_encoded_4.shape)
print("Data type of X_train_encoded_4:", type(X_train_encoded_4))
print("Data type of elements in X_train_encoded_4:", X_train_encoded_4.dtype)
print("Shape of X_test_encoded_4:", X_test_encoded_4.shape)
print("Data type of X_test_encoded_4:", type(X_test_encoded_4))
print("Data type of elements in X_test_encoded_4:", X_test_encoded_4.dtype)

# Scale numerical features
scaler_4 = StandardScaler()
X_train_scaled_4 = scaler_4.fit_transform(X_train_4[numerical_columns])
X_test_scaled_4 = scaler_4.transform(X_test_4[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_4:", X_train_scaled_4.shape)
print("Data type of elements in X_train_scaled_4:", X_train_scaled_4.dtype)
print("Shape of X_test_scaled_4:", X_test_scaled_4.shape)
print("Data type of elements in X_test_scaled_4:", X_test_scaled_4.dtype)

Shape of X_train_encoded_4: (40032, 3632)
Data type of X_train_encoded_4: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_4: float64
Shape of X_test_encoded_4: (10009, 3632)
Data type of X_test_encoded_4: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_4: float64
Shape of X_train_scaled_4: (40032, 1)
Data type of elements in X_train_scaled_4: float64
Shape of X_test_scaled_4: (10009, 1)
Data type of elements in X_test_scaled_4: float64


In [40]:
from scipy.sparse import hstack
# stage 1
X_train_processed = hstack([X_train_encoded, X_train_scaled]).astype(np.float32)
X_test_processed = hstack([X_test_encoded, X_test_scaled]).astype(np.float32)

# stage 2
X_train_processed_1 = hstack([X_train_encoded_1, X_train_scaled_1]).astype(np.float32)
X_test_processed_1 = hstack([X_test_encoded_1, X_test_scaled_1]).astype(np.float32)

# stage 3
X_train_processed_2 = hstack([X_train_encoded_2, X_train_scaled_2]).astype(np.float32)
X_test_processed_2 = hstack([X_test_encoded_2, X_test_scaled_2]).astype(np.float32)

# stage 4
X_train_processed_3 = hstack([X_train_encoded_3, X_train_scaled_3]).astype(np.float32)
X_test_processed_3 = hstack([X_test_encoded_3, X_test_scaled_3]).astype(np.float32)

# stage 5
X_train_processed_4 = hstack([X_train_encoded_4, X_train_scaled_4]).astype(np.float32)
X_test_processed_4 = hstack([X_test_encoded_4, X_test_scaled_4]).astype(np.float32)

In [41]:
# Dim 1
print("Data type of X_train_processed:", X_train_processed.dtype)
print("Data type of X_test_processed:", X_test_processed.dtype)
print("X_train_processed shape:", X_train_processed.shape)
print("X_test_processed shape:", X_test_processed.shape)

Data type of X_train_processed: float32
Data type of X_test_processed: float32
X_train_processed shape: (40032, 2196)
X_test_processed shape: (10009, 2196)


In [42]:
# Dim 2
print("Data type of X_train_processed_1:", X_train_processed_1.dtype)
print("Data type of X_test_processed_1:", X_test_processed_1.dtype)
print("X_train_processed_1 shape:", X_train_processed_1.shape)
print("X_test_processed_1 shape:", X_test_processed_1.shape)

Data type of X_train_processed_1: float32
Data type of X_test_processed_1: float32
X_train_processed_1 shape: (40032, 2254)
X_test_processed_1 shape: (10009, 2254)


In [43]:
# Dim 3
print("Data type of X_train_processed_2:", X_train_processed_2.dtype)
print("Data type of X_test_processed_2:", X_test_processed_2.dtype)
print("X_train_processed_2 shape:", X_train_processed_2.shape)
print("X_test_processed_2 shape:", X_test_processed_2.shape)

Data type of X_train_processed_2: float32
Data type of X_test_processed_2: float32
X_train_processed_2 shape: (40032, 2399)
X_test_processed_2 shape: (10009, 2399)


In [45]:
# Dim 4
print("Data type of X_train_processed_3:", X_train_processed_3.dtype)
print("Data type of X_test_processed_3:",X_test_processed_3.dtype)
print("X_train_processed_3 shape:", X_train_processed_3.shape)
print("X_test_processed_3 shape:", X_test_processed_3.shape)

Data type of X_train_processed_3: float32
Data type of X_test_processed_3: float32
X_train_processed_3 shape: (40032, 2944)
X_test_processed_3 shape: (10009, 2944)


In [46]:
# Dim 5
print("Data type of X_train_processed_4:", X_train_processed_4.dtype)
print("Data type of X_test_processed_4:", X_test_processed_4.dtype)
print("X_train_processed_4 shape:", X_train_processed_4.shape)
print("X_test_processed_4 shape:", X_test_processed_4.shape)

Data type of X_train_processed_4: float32
Data type of X_test_processed_4: float32
X_train_processed_4 shape: (40032, 3633)
X_test_processed_4 shape: (10009, 3633)
